# **Стабилизированный метод бисопряженных градиентов** (Biconjugate gradient stabilized method, BiCGStab)
BiCGStab является итерационным методом решения систем линейных алгебраических уравнений крыловского типа. Его можно применять к матрицам общего вида, в том числе над $\mathbb{C}$. В процессе выполнения он не использует для вычисления тяжелых для распараллеливания операций (транспонирования матриц, нахождения обратной матрицы и т.п.), а также не допускает накопления погрешностей округления, что делает его **стабильным** алгоритмом. К преимуществам так же можно добавить более высокую в общем случае скорость сходимости относительно более простых методов. Тем не менее, для некоторых видов СЛАУ BiCGStab может уступать им в скорости работы.

BiCGStab относится к классу так называемых **проекционных методов**. Основная идея методов данного класса: искать итерационным способом наилучшее приближение решения в некотором подпространстве пространства $\mathbb{R}^n$. В качестве такого подпространства используются пространства Крылова, порожденные матрицей системы.

Метод основан на построении биортогонального базиса $p_1, p_2, ..., p_k, ...$ пространства Крылова $K^k(A, r_0)$, где $A$ - матрица системы, $r_0$ - вектор невязки на нулевой итерации ($r_0 = b - Ax_0$), и дальнейшем вычислении поправки такой, что приближение на очередной итерации было бы ортогонально второму подпространству Крылова $K^k(A, \hat{r_0}$, где $\hat{r_0}$ - произвольный вектор, скалярное произведение которого с $r_0 \neq 0$ ($(r_0, \hat{r_0}) \neq 0)$. Базисные вектора строятся до достижения установленных критериев остановки (как правило, это достижение невязкой некоторого наперед заданного достаточно малого $\varepsilon$, что означает достижения некоторой окрестности решения), а каждое последующее приближение формируется как сумма приближения с предыдущей итерации и найденной поправки. 

Для решения СЛАУ вида $Ax = b$, где $A$ - комплексная матрица, стабилизированным методом бисопряженных градиентов может использоваться следующий алгоритм: 

**Перед итерационным процессом:** 
1. выбрать начальное приближение $x_0$ 
2. $r_0 = b - Ax_0$ 
3. выбрать произвольный вектор $\hat{r_0}$, т.ч. $(\hat{r_0}, r_0) \neq 0$, например, $\hat{r_0} = r_0$ (здесь $(x, y) = x^Ty)$ 
4. $\rho_0 = \alpha_0 = w_0 = 1$ 
5. $v_0 = p_0 = 0$
**$k$-я итерация метода** 
1. $\rho_k = (\hat{r_0}, r_{k-1})$ 
2. $\beta_k = \frac{\rho_k}{\rho_{k-1}} \cdot \frac{\alpha_{k-1}}{w_{k-1}}$ 
3. $p_k = r_{k-1} + \beta_k (p_{k-1} - w_{k-1} \cdot v_{k-1})$ 
4. $v_k = Ap_k$ 
5. $\alpha_k = \frac{\rho_k}{(\hat{r_0}, v_k)}$ 
6. $s_k = r_{k-1} - \alpha_k \cdot v_k$
7. $t_k = As_k$ 
8. $w_k = \frac{(t_k, s_k)}{(t_k, t_k)}$
9. $x_k =  x_{k-1} + \alpha_k \cdot p_k + w_k \cdot s_k$.
10. $r_k = s_k - w_k \cdot t_k$

**Критерии остановки:** 
1. ограниченное число итерацией ($k \leq k_{max}$) 
2. заданная невязка ($\frac{||r_k||}{||b||} < \varepsilon$) 
3. величина $|w_k| < \varepsilon_w$

In [1]:
import numpy as np
import numpy.linalg as linalg
import matplotlib.pyplot as plt

In [2]:
def BiCGStab(A, b, eps, max_steps= 1000, x0=None):
  # если начальное значение x0 не задано, то x0 = [0, ... 0].T
  if x0 is None:
    x0 = np.zeros_like(b)
  r = b - A.dot(x0)
  r_hat = r
  rho = alpha = w = 1
  v = p = 0
  steps = 0
  rho_prev = rho
  x = x0

  while linalg.norm(r) >= eps and steps < max_steps:
    steps += 1
    rho = r_hat.dot(r)
    beta = (rho / rho_prev) * (alpha / w)
    rho_prev = rho

    # обновляем р в 3 действия, зато не храним его предыдущее значение!
    p *= beta
    p -= beta * w * v
    p += r

    v = A.dot(p)
    alpha = rho / (r_hat.dot(v))
    h = x + alpha * p
    s = r - alpha * v
    if linalg.norm(s) <= eps:
      x = h
      break
    t = A.dot(s)
    w = t.dot(s) / t.dot(t)
    x = h + w * s
    r = s - w * t
  
  return [x, steps]

Проверим, работает ли вообще метод:

In [3]:
A = np.array([[1, 1, 5], [-3, 4, 0], [7, 3, -2]])
b = np.array([1, 1, 1]).T
x0 = np.array([1, 1, 1])
solution_first = BiCGStab(A, b, 1e-3, 1000, x0)
print(np.allclose(A.dot(solution_first[0]), b))

True


Сравним с решением, полученным при помощи `np.linalg.solve`

In [4]:
print(np.allclose(linalg.solve(A, b), solution_first[0]))

True


Реализуем также **обычный метод бисопряженных градиентов**. Сам алгоритм (для действительных матриц) выглядит так: \\
**Перед итерационным процессом:**
1. выбрать начальное приближение $x_0$
2. $r_0 = b - Ax_0$
3. $p_0 = r_0$
4. $z_0 = r_0$
5. $x_0 = r_0$
**$k$-я итерация алгоритма:**
1. $\alpha_k = \frac{(p_{k-1}, r_{k-1})}{(s_{k-1}, Az_{k-1})}$
2. $x_k = x_{k-1} + \alpha_k \cdot z_{k-1}$
3. $r_k = r_{k-1} - \alpha_kAz_{k-1}$
4. $p_k = p_{k-1} - \alpha_kA^Ts_{k-1}$
5. $\beta_k = \frac{(p_k, r_k)}{(p_{k-1}, r_{k-1})}$
6. $z_k = r_k + \beta_k \cdot z_{k-1}$
7. $s_k = p_k + \beta_k \cdot s_{k-1}$

In [5]:
def BiCG(A, b, eps, max_steps=1000, x0=None):
  if x0 is None:
    x0 = np.zeros_like(b)
  x = x0
  r = b - A.dot(x0)
  p = z = s = r
  steps = 0
  while np.linalg.norm(r) >= eps and steps < max_steps:
    alpha = p.dot(r) / s.dot(A.dot(z))
    x = x + alpha * z
    prev_mul = p.dot(r)
    r = r - alpha * A.dot(z)
    p = p - alpha * A.T.dot(s)
    beta = p.dot(r) / prev_mul
    z = r + beta * z
    s = p + beta * s
    steps += 1
  return [x, steps]

In [6]:
solution_second = BiCG(A, b, 1e-3, 1000, x0)
print(np.allclose(A.dot(solution_second[0]), b))

True


In [7]:
print(np.allclose(linalg.solve(A, b), solution_second[0]))

True


# Рассмотрим еще алгоритм GMRES - Generalized Minimum Residual Method

**GMRES** - это итерационный метод решения СЛАУ $Ax = b$ с произвольной матрицей $A$ - от нее требуется только невырожденность. Данный метод опирается на итерации Арнольди для нахождения минимизирующего невязку вектора из подпространства Крылова. Имеет существенный недостаток - поитерационное увеличение объема требуемой памяти, однако существует также версия алгоритма с рестартами, их использование позволяет существенно сократить объемы необходимой памяти. 

Основная идея - решение задачи наименьших квадратов на каждом шаге. Каждая итерация выглядит следующим образом: решение $x_k$ рассматривается в виде $x_0+Vy_k$, где столбцы $v_k$ матрицы $V$ находятся с помощью процесса Арнольди для построения ортонормированного базиса $(v_1,v_2,…,v_k)$ подпространства Крылова $K^k$($A,v_1$), вектор $y_k$ представляет собой решение системы $H_ky_k = \beta_ke_1$, где $H_k$ - верхняя матрица Хессенберга размера $k×k$, элементы которой формируются в процессе ортогонализации Арнольди, $\beta_k=∥r_0∥_2$, $r_0 = b − Ax_0$.

**Процесс ортогонализации Арнольди**: пусть $v_1$ - начальный вектор с единичной нормой, $k$ - размерность пр-ва. Каждый шаг процесса выглядит так:
1. $w = Av_j$
2. $h_{ij} = (w, v_i)$
3. $w = w - h_{ij}v_i$
4. $h_{(i + 1)j} = ||w||$
5. $v_{j+1} = \frac{w}{h_{(i+1)j}}$ 
($j = \overline{{1, k}}, i = \overline{{1, j}})$

**Минимизация нормы невязки** (решение задачи наименьших квадратов): невязка на $k$-й итерации имеет вид: $r_k = \beta e_1 - H_k y_k$. Очевидно, что $||r_k||_2 \rightarrow min$ при $H_k y_k = \beta e_1$ - осталось решить эту систему. В своей реализации для ее решения я буду использовать встроенную функцию из модуля `np.linalg`

![picture](https://drive.google.com/uc?export=view&id=1fTxEyVUJPHMyrYJ9kv5Piur2jGywW2VS)

In [56]:
def GMRes(A, b, x0, max_steps):
    r = b - np.array(np.dot(A, x0)).reshape(-1)
    x = []
    q = [0] * (max_steps)
    x.append(r)
    q[0] = r / np.linalg.norm(r)
    h = np.zeros((max_steps + 1, max_steps))
    for k in range(max_steps):
        y = np.array(np.dot(A, q[k])).reshape(-1)
        for j in range(k):
            h[j, k] = np.dot(q[j], y)
            y = y - h[j, k] * q[j]
        h[k + 1, k] = np.linalg.norm(y)
        if (h[k + 1, k] != 0 and k != max_steps - 1):
            q[k + 1] = y / h[k + 1, k]
        b = np.zeros(max_steps + 1)
        b[0] = np.linalg.norm(r)
        result = np.linalg.lstsq(h, b)[0]
        x.append(np.dot(np.array(q).T, result) + x0)
    return x

In [57]:
A = np.array([[1, 1], [3, -4]])
b = np.array([3, 2])
x0 = np.array([1, 2])
max_steps = 5
x = GMRes(A, b, x0, max_steps)
print(x[-1])

[2.01621042 0.95180204]


Решением системы является $x = 2, y = 1$, поэтому GMRES действительно приближает верное решение, однако точность намного хуже, чем у метода бисопряженных градиентов. Проверим на матрице большей размерности, используя реализацию из `scipy`

In [101]:
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import gmres
import time

A = np.random.rand(20, 20)
b = np.random.rand(20)
x0 = np.ones(20)
start = time.process_time()
x, exitCode = gmres(A, b)
end = time.process_time()
print(np.allclose(A.dot(x), b))
print('Время работы: ', end - start)

True
Время работы:  0.0014295039999971948


In [102]:
start = time.process_time()
solution_second = BiCG(A, b, 1e-3, 1000, x0)
end = time.process_time()
print(np.allclose(A.dot(solution_second[0]), b))
print('Время работы: ', end - start)

True
Время работы:  0.0023227500000011503


Видим, что BiCG требуется чуть больше времени, чтобы найти решение. При этом оба метода действительно хорошо приближают его.